In [13]:
from datetime import date, timedelta
from dotenv import load_dotenv
from ipywidgets import (
    interact, interactive, fixed, interact_manual, Layout
)
import ipywidgets as widgets
import json
import os
import time
import requests
import pandas as pd

load_dotenv()

TOKEN = os.getenv('TOKEN')

HEADERS = {"Authorization": f"Bearer {TOKEN}"}

GRAPHQL_URL = "https://api.github.com/graphql"

In [15]:
start_date = widgets.DatePicker(
    value=date.today() - timedelta(days=365),
    description='Start Date',
    disabled=False
)
end_date = widgets.DatePicker(
    value=date.today() - timedelta(days=1),
    description='End Date',
    disabled=False
)

def fdate(start, end):
    print(
        "\nstart date: {}, end date: {}\n".format(start, end)
    )

date_out = widgets.interactive_output(
    fdate, {
        "start": start_date,
        "end": end_date
    }
)

display(start_date, end_date, date_out)

DatePicker(value=datetime.date(2024, 7, 21), description='Start Date', step=1)

DatePicker(value=datetime.date(2025, 7, 20), description='End Date', step=1)

Output()

In [16]:
def get_unique_commit_authors_between_dates(since, until):
    """
    > Endpoint #1
    
    - Accepts: A start and end date.
    
    - Returns: A list of unique commit authors within the date constraints supplied.
    """
    print(f"\nSearching for commit authors since {since} until {until}\n")
    q = (
        "{ rateLimit { limit cost remaining resetAt } "
        " repository(name: \"OpenRA\", owner: \"OpenRA\") { "
        " ref(qualifiedName: \"bleed\") { target { ... on Commit { "
        f" history( first: 50 since: \"{since}\" until: \"{until}\") "
        " { pageInfo { hasNextPage startCursor endCursor } "
        " edges { node { author { name email }}}}}}}}}"
    )
    try:
        authors = set()
        while q is not None:
            r = requests.post(
                GRAPHQL_URL,
                json={"query": q},
                headers=HEADERS
            )
            print(r.status_code)
            if r.status_code == 429:
                time.sleep(60)
            if r.status_code != 200:
                print("uh oh")
                raise
            response = r.json()
            print(response)
            commits = response["data"]["repository"]["ref"]["target"]["history"]["edges"]
            for commit in commits:
                authors.add(commit["node"]["author"]["email"])
            if response["data"]["repository"]["ref"]["target"]["history"]["pageInfo"]["hasNextPage"] == True:
                after = response["data"]["repository"]["ref"]["target"]["history"]["pageInfo"]["endCursor"]
                print(after)
                q = (
                    "{ rateLimit { limit cost remaining resetAt } "
                    " repository(name: \"OpenRA\", owner: \"OpenRA\") { "
                    " ref(qualifiedName: \"bleed\") { target { ... on Commit { "
                    f' history( first: 50, after: "{after}", since: "{since}", until: "{until}") '
                    " { pageInfo { hasNextPage startCursor endCursor } "
                    " edges { node { author { name email }}}}}}}}}"
                )
            else:
                q = None
        print(f"\n*** Unique authors: \n\n{authors}\n\n")
        return authors
    except Exception as e:
        print(e)
        raise

unique_authors = get_unique_commit_authors_between_dates(
    f"{start_date.value}T00:00:00",
    f"{end_date.value}T00:00:00"
)


Searching for commit authors since 2024-07-21T00:00:00 until 2025-07-20T00:00:00

200
{'data': {'rateLimit': {'limit': 5000, 'cost': 1, 'remaining': 4980, 'resetAt': '2025-07-21T12:31:24Z'}, 'repository': {'ref': {'target': {'history': {'pageInfo': {'hasNextPage': True, 'startCursor': '53f959e4bf5140bd32c2d04e6de9fbc495e5433a 0', 'endCursor': '53f959e4bf5140bd32c2d04e6de9fbc495e5433a 49'}, 'edges': [{'node': {'author': {'name': 'Vapre', 'email': 'anvilvapre@users.noreply.github.com'}}}, {'node': {'author': {'name': 'Matthias Mailänder', 'email': 'matthias@mailaender.name'}}}, {'node': {'author': {'name': 'Matthias Mailänder', 'email': 'matthias@mailaender.name'}}}, {'node': {'author': {'name': 'Ashley Newson', 'email': 'ashleynewson@smartsim.org.uk'}}}, {'node': {'author': {'name': 'Ashley Newson', 'email': 'ashleynewson@smartsim.org.uk'}}}, {'node': {'author': {'name': 'darkademic', 'email': '41052878+darkademic@users.noreply.github.com'}}}, {'node': {'author': {'name': 'darkademic',

In [17]:
df = pd.DataFrame([(a,) for a in unique_authors], columns=["author"])
df
                  

,author
0,mustafaoyunda37@gmail.com
1,anvilvapre@users.noreply.github.com
2,gustas.kazukauskas@gmail.com
3,spamdog@gmail.com
4,porenut@centrum.sk
5,1872813635@qq.com
6,gregor@kalisnik.si
7,Smitty3228@gmail.com
8,ashleynewson@smartsim.org.uk
9,danmihai.stoian@gmail.com
